In [27]:
import numpy as np
import torch
from torch.utils import data # 和normal不同的地方
from d2l import torch as d2l

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [28]:
true_w = torch.tensor([2, -3.4])
true_b = torch.tensor(4.2)
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [29]:
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个pytorch迭代器
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train, generator=torch.Generator(device='cuda')) # 新版pytorch记得在dataloader后指定generator的device

batch_size = 10
data_iter = load_array((features, labels), batch_size)

print(next(iter(data_iter)))

[tensor([[-1.0662, -1.7531],
        [-0.9789,  0.5528],
        [-0.1894,  0.2043],
        [ 0.1258,  0.8813],
        [ 1.3177, -0.8540],
        [ 1.3989, -0.2982],
        [-2.2129,  0.4788],
        [-0.9084, -1.8034],
        [ 0.8475,  0.8483],
        [-0.6528, -0.2739]]), tensor([[ 8.0346],
        [ 0.3636],
        [ 3.1248],
        [ 1.4581],
        [ 9.7323],
        [ 8.0040],
        [-1.8546],
        [ 8.4904],
        [ 3.0057],
        [ 3.8348]])]


In [35]:
from torch import nn
'''
nn.Linear(m, n)定义了一个线性回归模型，第一层有m个unit，第二层有n个unit
nn.Sequential里面的参数是把一堆model按顺序叠在一起
'''
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01) # 为线性模型的weight和bias初始化参数
net[0].bias.data.fill_(0)

loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [36]:
num_epoch = 3
for epoch in range(num_epoch):
    for X, y in data_iter:
        l = loss(net(X),  y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print('epoch @{} loss : {}'.format(epoch + 1, float(l)))

epoch @1 loss : 0.00020432057499419898
epoch @2 loss : 0.00010260935960104689
epoch @3 loss : 0.00010328905773349106
